In [18]:
#import necessary libraries
import pandas as pd
import numpy as np
from pybaseball import batting_stats
import os

In [21]:
#load data from pybaseball (takes a long time to lad so lets toss the data to csv)
if os.path.exists('batting-stats-2000-2023.csv'):
    stats = pd.read_csv('batting-stats-2000-2023.csv', index_col=0)
else:
    stats = batting_stats(start_season=2000, end_season=2023, qual=200)
    stats.to_csv('batting-stats-2000-2023.csv')

In [35]:
stats = stats.groupby('IDfg', group_keys=False).filter(lambda x: len(x) > 1)

In [36]:
stats

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,...,O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,31,2,46,117,110,198,68,47,9,2,0,4,9,2,0.370,109.0,169.0,80.0,16.0,2398.0,1336.0,1062.0,5.0,0.0,0.0,0.324,0.077,4.21,0.582,0.799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139,347,47,171,189,273,111,1.05,70.0,135.0,249.0,105.0,106.0,87.0,64.0,83.0,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
2,1109,2001,Barry Bonds,SFG,36,153,476,664,156,49,32,2,73,129,137,177,35,93,9,2,0,5,13,3,0.328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.267,0.140,1.90,0.515,0.863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122,298,82,152,196,313,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,12.5
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,27,3,45,129,101,232,120,41,9,3,0,5,6,1,0.362,117.0,156.0,62.0,22.0,2425.0,1444.0,981.0,8.0,0.0,0.0,0.376,0.066,5.66,0.609,0.812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134,416,41,179,186,269,104,0.90,80.0,130.0,249.0,112.0,96.0,86.0,65.0,76.0,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
17,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,28,0,62,133,131,111,19,175,6,5,0,14,16,3,0.311,149.0,174.0,77.0,8.0,2906.0,1176.0,1730.0,11.0,0.0,0.0,0.159,0.251,0.63,0.425,0.686,...,0.224,0.654,0.427,0.452,0.826,0.722,0.471,24.5,NaN,128,200,113,137,175,251,117,0.97,87.0,116.0,324.0,118.0,91.0,84.0,46.0,83.0,159.0,95.8,14.9,106.0,0.262,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.6
3,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,22,1,45,111,90,148,61,58,10,2,0,7,7,0,0.341,101.0,152.0,81.0,21.0,2175.0,1141.0,1034.0,7.0,0.0,0.0,0.269,0.105,2.55,0.529,0.749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127,298,65,155,174,251,102,1.09,69.0,134.0,256.0,109.0,116.0,71.0,93.0,78.0,161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8079,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,12,0,16,40,49,41,2,192,7,4,0,5,2,0,0.168,111.0,110.0,59.0,6.0,2185.0,819.0,1366.0,6.0,2.0,0.0,0.079,0.368,0.21,0.243,0.296,...,0.273,0.622,0.451,0.471,0.772,0.682,0.509,21.8,NaN,67,95,169,76,71,77,80,0.99,94.0,108.0,114.0,99.0,94.0,109.0,75.0,108.0,102.0,89.3,15.1,27.0,0.096,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
7517,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,11,0,11,29,31,29,5,69,7,4,0,4,0,0,0.206,61.0,146.0,49.0,20.0,1323.0,437.0,886.0,3.0,0.0,0.0,0.080,0.191,0.42,0.283,0.343,...,0.361,0.686,0.531,0.618,0.873,0.790,0.522,22.8,-18.9,79,97,100,87,83,90,74,0.91,53.0,169.0,69.0,113.0,80.0,108.0,142.0,91.0,94.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
7669,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,16,0,11,36,42,75,0,177,4,2,0,10,0,1,0.159,78.0,114.0,48.0,15.0,2183.0,896.0,1287.0,0.0,0.0,0.0,0.151,0.357,0.42,0.292,0.277,...,0.247,0.608,0.420,0.498,0.785,0.698,0.480,21.0,NaN,62,187,199,90,68,78,82,1.03,74.0,129.0,96.0,126.0,71.0,98.0,100.0,90.0,122.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
8019,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,20,4,3,65,37,20,2,53,0,6,5,11,8,9,0.236,188.0,183.0,102.0,25.0,1860.0,579.0,1281.0,12.0,39.0,20.0,0.034,0.091,0.38,0.260,0.303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,41,56,79,71,42,87,1.02,94.0,106.0,15.0,88.0,96.0,123.0,110.0,107.0,76.0,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
#create column that shows a player's ops for the next season
def next_ops(player):
    player = player.sort_values('Season')
    player['Next_OPS'] = player['OPS'].shift(-1)
    return player

stats = stats.groupby('IDfg', group_keys=False).apply(next_ops)

In [38]:
stats[['Name','Season','OPS','Next_OPS']]

,Name,Season,OPS,Next_OPS
6411,Alfredo Amezaga,2006,0.664,0.682
5776,Alfredo Amezaga,2007,0.682,0.679
6054,Alfredo Amezaga,2008,0.679,NaN
2490,Garret Anderson,2000,0.827,0.792
3279,Garret Anderson,2001,0.792,0.871
...,...,...,...,...
3072,Paul O'Neill,2001,0.789,NaN
3816,Cal Ripken,2000,0.763,0.637
7413,Cal Ripken,2001,0.637,NaN
940,Quilvio Veras,2000,0.823,0.686


In [46]:
#drop all columns with null values except our next ops column
temp = stats['Next_OPS']
stats.dropna(axis='columns', inplace=True)
stats['Next_OPS'] = temp
stats

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,Bat,Rep,Pos,RAR,WAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,PH,WPA/LI,Clutch,BsR,Def,wSB,Age Rng,Off,Lg,TTO%,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,Events,L-WAR,Next_OPS
6411,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,9,3,3,42,19,33,4,46,3,1,7,5,20,12,0.260,0.087,0.122,0.72,0.332,0.332,0.664,0.072,0.294,0.296,-11.5,36,-12.4,11.9,1.2,11.9,1.1,5.9,75,-1.48,-6.93,5.45,-19.65,-1.96,0.98,26,-1.14,-0.37,2.8,8.8,-1.4,28 - 28,-9.6,0.8,0.217,96,98,74,97,75,42,97,0,1.1,0.682
5776,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,14,9,2,46,30,35,0,52,4,5,4,4,13,7,0.263,0.078,0.116,0.67,0.324,0.358,0.682,0.095,0.293,0.305,-9.9,45,-12.4,13.9,1.8,20.5,2.0,6.4,79,-2.52,-8.64,6.13,-18.88,-1.82,1.04,26,-0.76,-1.68,0.9,17.2,-0.8,29 - 29,-11.5,0.9,0.199,96,88,71,95,82,58,96,0,2.0,0.679
6054,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,13,5,3,41,32,19,1,47,3,0,4,6,8,2,0.264,0.056,0.139,0.40,0.312,0.367,0.679,0.103,0.303,0.301,-7.5,33,-9.6,10.3,1.6,11.7,1.2,6.8,77,-0.59,-6.14,5.55,-8.48,-0.82,1.05,25,-0.96,0.39,2.4,8.0,0.5,30 - 30,-7.1,0.6,0.205,99,62,81,92,86,65,100,0,1.2,NaN
2490,2,2000,Garret Anderson,ANA,28,159,647,681,185,107,40,3,35,92,117,24,5,87,0,9,1,21,7,6,0.286,0.035,0.128,0.28,0.307,0.519,0.827,0.233,0.281,0.345,2.3,92,-0.8,22.0,0.4,23.8,2.2,3.8,99,-2.17,-14.93,12.76,-9.29,-0.82,1.11,1,-0.32,-1.63,-1.3,1.4,-1.3,28 - 28,-2.1,2.5,0.214,104,37,82,88,117,139,93,0,2.2,0.792
3279,2,2001,Garret Anderson,ANA,29,161,672,704,194,125,39,2,28,83,123,27,4,100,0,5,0,12,13,6,0.289,0.038,0.142,0.27,0.314,0.478,0.792,0.189,0.302,0.334,4.3,92,3.4,22.0,-7.6,27.2,2.7,4.0,104,1.55,-13.88,15.43,13.57,1.40,1.08,0,1.20,0.23,0.0,-0.6,0.0,29 - 29,3.4,2.5,0.220,108,46,86,94,111,117,102,0,2.7,0.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,1009818,2001,Paul O'Neill,NYY,38,137,510,563,136,81,33,1,21,77,70,48,4,59,2,3,0,20,22,3,0.267,0.085,0.105,0.81,0.330,0.459,0.789,0.192,0.266,0.337,4.7,75,5.1,17.6,-6.3,6.4,0.6,5.5,107,0.99,-10.52,11.51,-0.08,-0.02,1.04,4,0.47,0.49,3.1,-21.3,3.1,38 - 38,8.2,2.0,0.227,100,103,64,99,107,119,89,0,0.6,NaN
3816,1010978,2000,Cal Ripken,BAL,39,83,309,339,79,48,16,0,15,43,56,23,0,37,3,4,0,10,0,0,0.256,0.068,0.109,0.62,0.310,0.453,0.763,0.197,0.245,0.328,-3.8,41,-3.7,11.0,0.0,13.5,1.3,1.5,92,0.12,-6.27,6.39,-3.07,-0.26,0.98,0,-0.04,0.16,0.0,5.0,0.0,39 - 39,-3.6,1.2,0.221,93,71,70,89,102,117,81,0,1.3,0.637
7413,1010978,2001,Cal Ripken,BAL,40,128,477,516,114,84,16,0,14,43,68,26,1,63,2,9,2,15,0,2,0.239,0.050,0.122,0.41,0.276,0.361,0.637,0.122,0.244,0.275,-22.7,41,-21.4,16.1,0.2,-5.3,-0.5,1.3,67,-2.03,-9.86,7.82,-17.45,-1.78,1.02,3,-2.18,0.19,-0.9,-0.8,-0.9,40 - 40,-22.3,1.8,0.200,89,61,74,83,84,75,82,0,-0.5,NaN
940,1013404,2000,Quilvio Veras,ATL,29,84,298,364,92,72,15,0,5,56,37,51,0,50,5,4,6,8,25,12,0.309,0.140,0.137,1.02,0.413,0.409,0.823,0.101,0.352,0.374,10.5,58,9.2,11.8,1.2,28.2,2.7,4.9,119,0.91,-6.49,7.41,13.17,1.36,1.04,2,1.44,-0.56,-0.5,7.2,-0.5,29 - 29,8.7,0.5,0.291,113,140,85,118,92,58,117,0,2.7,0.686


In [48]:
#lets modify some string types to help with ml
stats.dtypes[stats.dtypes == 'object']

Name       object
Team       object
Age Rng    object
dtype: object

In [50]:
#no need for age range
stats.drop(columns='Age Rng', inplace=True)

In [59]:
#convert team name to team number
numbers, teams = stats['Team'].factorize()
stats['Team_Num'] = numbers
stats

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,Bat,Rep,Pos,RAR,WAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,PH,WPA/LI,Clutch,BsR,Def,wSB,Off,Lg,TTO%,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,Events,L-WAR,Next_OPS,Team_Num
6411,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,9,3,3,42,19,33,4,46,3,1,7,5,20,12,0.260,0.087,0.122,0.72,0.332,0.332,0.664,0.072,0.294,0.296,-11.5,36,-12.4,11.9,1.2,11.9,1.1,5.9,75,-1.48,-6.93,5.45,-19.65,-1.96,0.98,26,-1.14,-0.37,2.8,8.8,-1.4,-9.6,0.8,0.217,96,98,74,97,75,42,97,0,1.1,0.682,0
5776,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,14,9,2,46,30,35,0,52,4,5,4,4,13,7,0.263,0.078,0.116,0.67,0.324,0.358,0.682,0.095,0.293,0.305,-9.9,45,-12.4,13.9,1.8,20.5,2.0,6.4,79,-2.52,-8.64,6.13,-18.88,-1.82,1.04,26,-0.76,-1.68,0.9,17.2,-0.8,-11.5,0.9,0.199,96,88,71,95,82,58,96,0,2.0,0.679,0
6054,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,13,5,3,41,32,19,1,47,3,0,4,6,8,2,0.264,0.056,0.139,0.40,0.312,0.367,0.679,0.103,0.303,0.301,-7.5,33,-9.6,10.3,1.6,11.7,1.2,6.8,77,-0.59,-6.14,5.55,-8.48,-0.82,1.05,25,-0.96,0.39,2.4,8.0,0.5,-7.1,0.6,0.205,99,62,81,92,86,65,100,0,1.2,NaN,0
2490,2,2000,Garret Anderson,ANA,28,159,647,681,185,107,40,3,35,92,117,24,5,87,0,9,1,21,7,6,0.286,0.035,0.128,0.28,0.307,0.519,0.827,0.233,0.281,0.345,2.3,92,-0.8,22.0,0.4,23.8,2.2,3.8,99,-2.17,-14.93,12.76,-9.29,-0.82,1.11,1,-0.32,-1.63,-1.3,1.4,-1.3,-2.1,2.5,0.214,104,37,82,88,117,139,93,0,2.2,0.792,1
3279,2,2001,Garret Anderson,ANA,29,161,672,704,194,125,39,2,28,83,123,27,4,100,0,5,0,12,13,6,0.289,0.038,0.142,0.27,0.314,0.478,0.792,0.189,0.302,0.334,4.3,92,3.4,22.0,-7.6,27.2,2.7,4.0,104,1.55,-13.88,15.43,13.57,1.40,1.08,0,1.20,0.23,0.0,-0.6,0.0,3.4,2.5,0.220,108,46,86,94,111,117,102,0,2.7,0.871,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,1009818,2001,Paul O'Neill,NYY,38,137,510,563,136,81,33,1,21,77,70,48,4,59,2,3,0,20,22,3,0.267,0.085,0.105,0.81,0.330,0.459,0.789,0.192,0.266,0.337,4.7,75,5.1,17.6,-6.3,6.4,0.6,5.5,107,0.99,-10.52,11.51,-0.08,-0.02,1.04,4,0.47,0.49,3.1,-21.3,3.1,8.2,2.0,0.227,100,103,64,99,107,119,89,0,0.6,NaN,17
3816,1010978,2000,Cal Ripken,BAL,39,83,309,339,79,48,16,0,15,43,56,23,0,37,3,4,0,10,0,0,0.256,0.068,0.109,0.62,0.310,0.453,0.763,0.197,0.245,0.328,-3.8,41,-3.7,11.0,0.0,13.5,1.3,1.5,92,0.12,-6.27,6.39,-3.07,-0.26,0.98,0,-0.04,0.16,0.0,5.0,0.0,-3.6,1.2,0.221,93,71,70,89,102,117,81,0,1.3,0.637,27
7413,1010978,2001,Cal Ripken,BAL,40,128,477,516,114,84,16,0,14,43,68,26,1,63,2,9,2,15,0,2,0.239,0.050,0.122,0.41,0.276,0.361,0.637,0.122,0.244,0.275,-22.7,41,-21.4,16.1,0.2,-5.3,-0.5,1.3,67,-2.03,-9.86,7.82,-17.45,-1.78,1.02,3,-2.18,0.19,-0.9,-0.8,-0.9,-22.3,1.8,0.200,89,61,74,83,84,75,82,0,-0.5,NaN,27
940,1013404,2000,Quilvio Veras,ATL,29,84,298,364,92,72,15,0,5,56,37,51,0,50,5,4,6,8,25,12,0.309,0.140,0.137,1.02,0.413,0.409,0.823,0.101,0.352,0.374,10.5,58,9.2,11.8,1.2,28.2,2.7,4.9,119,0.91,-6.49,7.41,13.17,1.36,1.04,2,1.44,-0.56,-0.5,7.2,-0.5,8.7,0.5,0.291,113,140,85,118,92,58,117,0,2.7,0.686,3


In [61]:
#drop rows with null next_ops values (keep copy for later in case)
complete_stats = stats.copy()
stats.dropna(inplace=True)

In [ ]:
#import machine learning libraries / functions
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit